# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Модели-с-базовыми-гиперпараметрами" data-toc-modified-id="Модели-с-базовыми-гиперпараметрами-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модели с базовыми гиперпараметрами</a></span><ul class="toc-item"><li><span><a href="#CatBoostRegressor" data-toc-modified-id="CatBoostRegressor-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>CatBoostRegressor</a></span></li><li><span><a href="#LGBMRegressor" data-toc-modified-id="LGBMRegressor-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>LGBMRegressor</a></span></li><li><span><a href="#LinearRegression" data-toc-modified-id="LinearRegression-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>LinearRegression</a></span></li></ul></li><li><span><a href="#Модели-с-подбраными-гиперпараметрами" data-toc-modified-id="Модели-с-подбраными-гиперпараметрами-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модели с подбраными гиперпараметрами</a></span><ul class="toc-item"><li><span><a href="#CatBoostRegressor" data-toc-modified-id="CatBoostRegressor-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>CatBoostRegressor</a></span></li><li><span><a href="#LGBMRegressor" data-toc-modified-id="LGBMRegressor-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>LGBMRegressor</a></span></li></ul></li></ul></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Описание данных

**Признаки**

* *`DateCrawled`* — дата скачивания анкеты из базы

* *`VehicleType`* — тип автомобильного кузова

* *`RegistrationYear`* — год регистрации автомобиля

* *`Gearbox`* — тип коробки передач

* *`Power`* — мощность (л. с.)

* *`Model`* — модель автомобиля

* *`Kilometer`* — пробег (км)

* *`RegistrationMonth`* — месяц регистрации автомобиля

* *`FuelType`* — тип топлива

* *`Brand`* — марка автомобиля

* *`NotRepaired`* — была машина в ремонте или нет

* *`DateCreated`* — дата создания анкеты

* *`NumberOfPictures`* — количество фотографий автомобиля

* *`PostalCode`* — почтовый индекс владельца анкеты (пользователя)

* *`LastSeen`* — дата последней активности пользователя

**Целевой признак**

* *`Price`* — цена (евро)

## Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import plotly.graph_objs as go
from plotly.offline import iplot
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/datasets/autos.csv')

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

Удалим дубликаты:

In [ ]:
df.drop_duplicates(inplace = True)

Удалим столбцы с датами:

In [ ]:
df = df.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis = 1)

Создадим список со столбцами с количественными значениями:

In [ ]:
int_feat = ['Price', 'RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']

Посмотрим категории в категориальных столбцах:

In [ ]:
for col in df.columns:
    if col not in int_feat:
        print(col)
        print(df[col].value_counts())
        print()

Напишем функцию для просмотра распределений количественных столбцов:

In [ ]:
def hist_plotly(df, hist_type):
    traces = []
    for feat in int_feat:
        traces.append(
            hist_type(x = df[feat],
                   name = feat,
                   visible=(feat == 'Price')
                        )
        )
    
    layout = go.Layout(
        title='Распределение событий',
        template='plotly_white',
        legend=dict(
            orientation="h"),
        updatemenus=list([
            dict(
                type="buttons",
                x=-0.05,
                y=1.05,
                direction="right",
                showactive=True,
                xanchor="left",
                buttons=list([
                    dict(
                        args=['visible', [True]*1 + [False]*6],
                        label=int_feat[0],
                        method='restyle'
                    ),
                    dict(
                        args=['visible', [False]*1 + [True]*1 + [False]*5],
                        label=int_feat[1],
                        method='restyle'
                    ),
                    dict(
                        args=['visible', [False]*2 + [True]*1 + [False]*4],
                        label=int_feat[2],
                        method='restyle'
                    ),
                    dict(
                        args=['visible', [False]*3 + [True]*1 + [False]*3],
                        label=int_feat[3],
                        method='restyle'
                    ),
                    dict(
                        args=['visible', [False]*4 + [True]*1 + [False]*2],
                        label=int_feat[4],
                        method='restyle'
                    )
                ]),
            )
        ]),
    )

    fig = {'data': traces, 'layout': layout}
    iplot(fig, show_link=False)

Посмотрим распределение в исходных данных:

In [ ]:
hist_plotly(df, go.Box) # go.Box, go.Histogram

Напишем функцию для очистки данных по квантилям:

In [ ]:
def filter_data(df, column):
    stat = df[column].describe()
    iqr = stat[6] - stat[4] 
    left_whisker = round(stat[4] - 1.5 * iqr, 2)
    right_whisker = round(stat[6] + 1.5 * iqr, 2)
    if left_whisker < stat[3]: left_whisker = stat[3] 
    if right_whisker > stat[7]: right_whisker = stat[7]
    return [left_whisker, right_whisker]

Применим функцию к нашим данным, результаты сохраним в переменной *`train_df_clear`*:

In [ ]:
train_df_clear = pd.DataFrame()
for row in df:
    try:
        train_df_clear[row] = df.loc[(df[row] >= filter_data(df, row)[0]) & (df[row] <= filter_data(df, row)[1])][row]
    except:
        train_df_clear[row] = df[row]

Присвоим категориальным данным тип *`category`*:

In [ ]:
train_df_clear[['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']] = train_df_clear[['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']].astype('category')

In [ ]:
train_df_clear.dropna(inplace=True)
train_df_clear.head(5)

Построим гистограммы распределения для подготовленных данных:

In [ ]:
hist_plotly(train_df_clear, go.Box) # go.Box, go.Histogram

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(train_df_clear.corr(), annot=True, fmt='.2f')
plt.show()

**Выводы:**

1. Категориальные данные не содержат ошибок, но содержат пропуски. Объем данных позволят нам удалить их.
2. Количественные данные имеют много выбросов, мы удалили их по квантилям.
3. В данных всего несколько дубликатов.
4. Существует небольшая корреляция в данных между годом регистрации/ценой, мощностью/ценой и брендом/моделью. Корреляция незначительная и не должна повлиять на качество обучения моделей. 

## Обучение моделей

### Модели с базовыми гиперпараметрами

Выделим из данных целевой признак:

In [ ]:
features = train_df_clear.drop(['Price'], axis = 1)
target = train_df_clear['Price']

Разделим данные на обучающую и тестовую выборки:

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, 
    target, 
    test_size = 0.25, 
    random_state = 12345)

Сформируем список категориальных столбцов:

In [ ]:
category_feat = features.select_dtypes('category').columns.tolist()

#### CatBoostRegressor

Исследуем модель **CatBoostRegressor** без подбора гиперпараметров:

In [ ]:
model_cat = CatBoostRegressor(loss_function='RMSE', random_state=12345)

start = time.time()

cat = model_cat.fit(features_train,
                    target_train,
                    cat_features = category_feat,
                    verbose=False
                   )

finish = time.time()
time_cat = finish - start

predict_cat = model_cat.predict(features_test)
rmse_cat = np.sqrt(mean_squared_error(predict_cat, target_test))

print('RMSE: ', rmse_cat)
print('Time: ', time_cat)

#### LGBMRegressor

Исследуем модель **LGBMRegressorr** без подбора гиперпараметров:

In [ ]:
model_lgb = lgb.LGBMRegressor(random_state=12345)

start = time.time()

lgb = model_lgb.fit(features_train,
                    target_train,
                    eval_metric = 'RMSE',
                    categorical_feature = category_feat)

finish = time.time()
time_lgb = finish - start

predict_lgb = model_lgb.predict(features_test, num_iteration = model_lgb.best_iteration_)
rmse_lgb = np.sqrt(mean_squared_error(predict_lgb, target_test))

print('RMSE: ', rmse_lgb)
print('Time: ', time_lgb)

Появляется ошибка:

*categorical_feature in Dataset is overridden.
New categorical_feature is ['Brand', 'FuelType', 'Gearbox', 'Model', 'NotRepaired', 'VehicleType']*

но я не нашел пути ее решения и не обнаружил ее влияния на обучение.

#### LinearRegression

Закодируем категориальные столбцы с помощью *`OrdinalEncoder()`*. Пропуски заменим на **None**.

In [ ]:
df_encoder = train_df_clear
df_encoder[category_feat] = df_encoder[category_feat].replace(np.nan, 'None')
df_encoder[category_feat] = pd.DataFrame(OrdinalEncoder().fit_transform(train_df_clear[category_feat]), columns=category_feat)
df_encoder.dropna(inplace=True)
df_encoder.head(5)

Выделим целевой признак и разделим данные на выборки:

In [ ]:
features_encoder = df_encoder.drop(['Price'], axis = 1)
target_encoder = df_encoder['Price']

In [ ]:
features_train_encoder, features_test_encoder, target_train_encoder, target_test_encoder = train_test_split(
    features_encoder, target_encoder, test_size = 0.25, random_state = 12345)

Исследуем модель **LinearRegression**:

In [ ]:
model_lr = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller), ("linear_regression", model_lr)])

start = time.time()

pipeline.fit(features_train_encoder, target_train_encoder)

finish = time.time()
time_lr = finish - start

predict_lr = pipeline.predict(features_test_encoder)
rmse_lr = np.sqrt(mean_squared_error(predict_lr, target_test_encoder))

print('RMSE: ', rmse_lr)
print('Time: ', time_lr)

**Вывод**

Модели **CatBoostRegressor** и **LGBMRegressor** показывают практически идентичные результаты без подбора гиперпараметров - RMSE 1298 и 1266 соответвенно за приблизительно одинаковое время. Модель **LinearRegression** обучается значительно быстрее, но показывает скромные RMSE 2154.

### Модели с подбраными гиперпараметрами

#### CatBoostRegressor

Исследуем модель **CatBoostRegressor** с подобором гиперпараметров (исследованные диапазоны параметров вынесены за #):

In [ ]:
param_cat = {
    'learning_rate': [0.08], # 0.01-1
    'iterations': [1000], # 100-1000
    'depth': [12] # 10-20
}

cv_cat = KFold(n_splits=3,
               shuffle=False)

clf_cat = GridSearchCV(model_cat, 
                        param_cat,
                        cv=cv_cat,
                        scoring='neg_mean_squared_error')

In [ ]:
clf_cat.fit(features_train,
            target_train,
            cat_features = category_feat,
            verbose=False)

print('Best Params:', clf_cat.best_params_)

In [ ]:
clf_cat_best = clf_cat.best_estimator_

start = time.time()

clf_cat_best.fit(features_train,
                 target_train,
                 cat_features = category_feat,
                 verbose=False)

finish = time.time()
time_clf_cat = finish - start

predict_clf_cat = clf_cat_best.predict(features_test)
rmse_clf_cat = np.sqrt(mean_squared_error(predict_clf_cat, target_test))

print('RMSE: ', rmse_clf_cat)
print('Time: ', time_clf_cat)

#### LGBMRegressor

Исследуем модель **LGBMRegressor** с подобором гиперпараметров:

In [ ]:
param_lgb = {
    'learning_rate': [0.08], #0.01-1
    'n_estimators': [1000], # 10-1000
    'max_depth': [12], # 1-20
}

cv_lgb = KFold(n_splits=3,
               shuffle=False)

clf_lgb = GridSearchCV(model_lgb, 
                        param_lgb,
                        cv = cv_lgb,
                        scoring = 'neg_mean_squared_error')

In [ ]:
clf_lgb.fit(features_train,
             target_train,
             eval_metric = 'RMSE',
             categorical_feature = category_feat)

print('Best Params:', clf_lgb.best_params_)

In [ ]:
clf_lgb_best = clf_lgb.best_estimator_

start = time.time()

clf_lgb_best.fit(features_train,
                  target_train,
                  eval_metric='RMSE')

finish = time.time()
time_clf_lgb = finish - start

predict_clf_lgb = clf_lgb_best.predict(features_test)
rmse_clf_lgb = np.sqrt(mean_squared_error(predict_clf_lgb, target_test))

print('RMSE: ', rmse_clf_lgb)
print('Time: ', time_clf_lgb)

**Вывод**

Благодаря подбору гиперпараметров удалось значительно снизить время обучения, при сохранении значений RMSE.

## Анализ моделей

In [ ]:
index = ['LinearRegression', 'CatBoostRegressor', 'LGBMRegressor', 'CatBoostRegressor optimized', 'LGBMRegressor optimized']
data = {'RMSE, евро':[rmse_lr, rmse_cat, rmse_lgb, rmse_clf_cat, rmse_clf_lgb],
        'Время обучения, сек':[time_lr, time_cat, time_lgb, time_clf_cat, time_clf_lgb,]}
scores_data = pd.DataFrame(data = data, index = index)
scores_data

**Вывод**

Подбирая гиперпараметры, мы смогли немного улучшить значение RMSE, но при этом скорость обучения выросла приблизительно в 5 раз.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей